## DATA FROM THE WEB

In [1]:
from bs4 import BeautifulSoup
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests as rq
import seaborn as sns
import re
sns.set_context('notebook')

## Function getParamsRequestValue

#### TODO comment

In [2]:
def getParamsRequestValue(i_reportmodel_num, i_reportmodel_xsl_num, section, year, semester): 
    r = rq.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter', params={'ww_i_reportmodel' : i_reportmodel_num})
    parameters_map = { 'ww_x_UNITE_ACAD' : section, 'ww_x_PERIODE_ACAD' : year, 'ww_x_PERIODE_PEDAGO' : semester }
    
    soup = BeautifulSoup(r.text, 'html.parser')
    
    ps = {key: next((option.attrs['value'] for option in soup.find('select', {'name': key }) if option.text==value), None) for key, value in parameters_map.items()}
    o = {'ww_i_reportmodel' : i_reportmodel_num, 'ww_b_list' : 1, 'ww_i_reportModelXsl' : i_reportmodel_xsl_num, 'ww_x_HIVERETE' : 'null', 'dummy' : 'ok'}
    d = {**o, **ps}
    r = rq.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter', params=d)
    soup = BeautifulSoup(r.text, 'html.parser')
    for p in soup.find_all('a'):
        if(p.text==', '.join([section, year, semester])):
            arr = re.search('\(\'(.+?)\'\)', p['onclick']).group(1).split("=")
            d[arr[0]] = int(arr[1])
                  
    return d

#getParamsRequestValue(133685247, 133685270, 'Informatique', '2007-2008', 'Bachelor semestre 1')

### Initialize the value of reportmodel and Section, years and semesters we want.

In [5]:
ww_i_reportmodel_num=133685247
ww_i_reportModelXsl_num=133685270
section = 'Informatique'
bachelor_semesters = ['Bachelor semestre 1', 'Bachelor semestre 2', 'Bachelor semestre 3', 'Bachelor semestre 4', 'Bachelor semestre 5', 'Bachelor semestre 6']
years_semesters = {'2007-2008': ['Bachelor semestre 1', 'Bachelor semestre 2'], '2008-2009': ['Bachelor semestre 1', 'Bachelor semestre 2', 'Bachelor semestre 3', 'Bachelor semestre 4'] , '2009-2010': bachelor_semesters, '2010-2011': bachelor_semesters, '2011-2012': bachelor_semesters, '2012-2013':bachelor_semesters, '2013-2014': bachelor_semesters, '2014-2015': bachelor_semesters, '2015-2016': ['Bachelor semestre 5', 'Bachelor semestre 6'], '2016-2017': ['Bachelor semestre 5', 'Bachelor semestre 6'] }

In [6]:
for year,semesters in years_semesters.items():
    for semester in semesters:
        payload = getParamsRequestValue(ww_i_reportmodel_num, ww_i_reportModelXsl_num, section, year, semester)
        r = rq.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?', params=payload)
        soup = BeautifulSoup(r.text, 'html.parser')
        # Print the title of each table we will take into account
        print(soup.table.tr.font.string)


Informatique, 2011-2012, Bachelor semestre 1
Informatique, 2011-2012, Bachelor semestre 2
Informatique, 2011-2012, Bachelor semestre 3
Informatique, 2011-2012, Bachelor semestre 4
Informatique, 2011-2012, Bachelor semestre 5
Informatique, 2011-2012, Bachelor semestre 6
Informatique, 2015-2016, Bachelor semestre 5
Informatique, 2015-2016, Bachelor semestre 6
Informatique, 2016-2017, Bachelor semestre 5
Informatique, 2016-2017, Bachelor semestre 6
Informatique, 2008-2009, Bachelor semestre 1
Informatique, 2008-2009, Bachelor semestre 2
Informatique, 2008-2009, Bachelor semestre 3
Informatique, 2008-2009, Bachelor semestre 4
Informatique, 2007-2008, Bachelor semestre 1
Informatique, 2007-2008, Bachelor semestre 2
Informatique, 2009-2010, Bachelor semestre 1
Informatique, 2009-2010, Bachelor semestre 2
Informatique, 2009-2010, Bachelor semestre 3
Informatique, 2009-2010, Bachelor semestre 4
Informatique, 2009-2010, Bachelor semestre 5
Informatique, 2009-2010, Bachelor semestre 6
Informatiq

In [190]:
attributs = soup.find_all('th')
for attr in attributs:
    print(attr.string)   
print(sr)

None
Civilité
Nom Prénom
Orientation Bachelor
Orientation Master
Spécialisation
Filière opt.
Mineur
Statut
Type Echange
Ecole Echange
No Sciper
0    test
dtype: object


In [7]:
df = pd.DataFrame()
i=0
ppl_attributes = soup.find_all(style="white-space:nowrap")
for ppl_attr in ppl_attributes:
    df.loc()
    i=i+1
    
    #print(str(ppl_attr.string) + str(i))
    